In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

config_file = "../../../config/agam.2025.03.05.yaml"
input_nb = "test.ipynb"
output_nb = "../../../results/dev/test.ipynb"

In [ ]:
import json
from uuid import uuid4
from selection_atlas.setup import AtlasSetup
from selection_atlas.page_utils import AtlasPageUtils

# Initialise the atlas setup.
setup = AtlasSetup(config_file)
page_utils = AtlasPageUtils(setup=setup)

In [ ]:
def get_markdown_output(cell):
    """Get markdown outputs from a code cell."""
    cell_type = cell.get("cell_type")
    if cell_type == "code":
        outputs = cell.get("outputs", [])
        if len(outputs) == 1:
            output = outputs[0]
            output_type = output.get("output_type")
            if output_type == "execute_result":
                data = output.get("data", {})
                return data.get("text/markdown")

In [ ]:
def create_markdown_cell(source):
    """Create a new markdown cell with the given source content."""
    return {
        "cell_type": "markdown",
        "id": str(uuid4()),
        "metadata": {},
        "source": source,
    }

In [ ]:
def prepare_notebook(input_nb, output_nb):
    """Prepare a notebook for inclusion in a Jupyter book."""

    # Read in the input notebook.
    with open(input_nb, mode="r", encoding="utf-8") as f:
        my_nb = json.loads(f.read())

    # Process cells.
    cells = my_nb["cells"]
    for i, cell in enumerate(cells):
        # Hide papermill injected parameters cells.
        metadata = cell["metadata"]
        if "tags" in metadata:
            tags = metadata["tags"]
            if "injected-parameters" in tags:
                tags.append("remove-input")
                continue

        # Replace code cells generating markdown outputs with markdown cells.
        md = get_markdown_output(cell)
        if md:
            cell = create_markdown_cell(md)
            cells[i] = cell

    # Write out the processed notebook.
    with open(output_nb, "w") as f:
        json.dump(my_nb, f)

In [ ]:
prepare_notebook(
    input_nb=input_nb,
    output_nb=output_nb,
)